# Classificação - Base exame diabetes

In [25]:
import pandas as pd


df_train = pd.read_csv("data/diabete/Dados_Treino.csv")
df_test = pd.read_csv("data/diabete/Dados_Teste.csv")
df_template = pd.read_csv("data/diabete/Template_Submissao.csv")


In [26]:
# df_steam.dtypes

df_train.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
1,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
2,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,...,337.0,337.0,0,0,0,0,0,0,0,0
3,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
4,182697,83,67.0,27.555611,0,Caucasian,M,190.5,Direct Admit,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,1


In [27]:
df_test.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,231640,42,35.0,29.060607,0,Caucasian,F,169.0,Emergency Department,Accident & Emergency,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,220057,100,84.0,23.128938,0,Caucasian,F,170.2,Emergency Department,Accident & Emergency,...,NaN,NaN,NaN,0,0,0,0,0,0,0
2,238466,128,61.0,42.873376,0,Caucasian,M,175.2,Emergency Department,Accident & Emergency,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,214794,77,86.0,25.795461,0,Caucasian,F,162.6,NaN,Floor,...,NaN,NaN,NaN,0,0,0,0,0,0,0
4,255804,110,38.0,41.868512,1,African American,F,170.0,Operating Room,Operating Room / Recovery,...,202.0,202.0,202.0,0,0,0,0,0,0,0


In [28]:
df_template.head()

,encounter_id,diabetes_mellitus
0,231640,NaN
1,220057,NaN
2,238466,NaN
3,214794,NaN
4,255804,NaN


#### Remove as linhas que estão vazias -> NaN

In [35]:
df_template = df_template[(df_template['diabetes_mellitus'].notna()) & (df_template['diabetes_mellitus'] == True)]
df_template

# Todos os valores são Nan, o dataset fica vazio

,encounter_id,diabetes_mellitus


In [41]:
df_train_columns = df_train.columns
len(df_train_columns)

180

In [42]:
df_test_columns = df_test.columns
len(df_test_columns)

179

In [44]:
df_train_columns.difference(df_test_columns)


Index(['diabetes_mellitus'], dtype='object')

In [47]:
df_train['diabetes_mellitus']

0         1
1         1
2         0
3         0
4         1
         ..
117136    0
117137    0
117138    1
117139    0
117140    0
Name: diabetes_mellitus, Length: 117141, dtype: int64

In [54]:
df_train_null = df_train.isnull().sum()
df_train_null.sort_values()

encounter_id              0
lymphoma                  0
leukemia                  0
immunosuppression         0
hepatic_failure           0
                      ...  
h1_lactate_max       106603
h1_albumin_max       107055
h1_albumin_min       107055
h1_bilirubin_max     107832
h1_bilirubin_min     107832
Length: 180, dtype: int64

In [57]:
list(df_train.columns)

['encounter_id',
 'hospital_id',
 'age',
 'bmi',
 'elective_surgery',
 'ethnicity',
 'gender',
 'height',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_id',
 'icu_stay_type',
 'icu_type',
 'pre_icu_los_days',
 'readmission_status',
 'weight',
 'albumin_apache',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'apache_post_operative',
 'arf_apache',
 'bilirubin_apache',
 'bun_apache',
 'creatinine_apache',
 'fio2_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'intubated_apache',
 'map_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'ventilated_apache',
 'wbc_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp

In [64]:
df_test['cirrhosis'].value_counts()

cirrhosis
0    12812
1      204
Name: count, dtype: int64